# IoT and Big Data Fall 2025
# Neuro fuzzy system for smart precision farming
# Sharmetov Nurdaulet, 24MD0387
### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

### Since we don't have a ready-on dataset, we generated synthetic data, but with certain logic, i.e if the temperature is hot, we expect less humidity and etc. 

# Generating synthetic dataset: configuration

In [2]:
N = 2000                              # Number of samples
interval_minutes = 10                 # Sensor frequency (every 10 minutes)
start_time = datetime(2024, 1, 1, 0, 0)

# Environmental baseline values
soil_moisture = 45.0                 # % VWC
temperature = 24.0                   # °C
humidity = 55.0                      # %
rainfall_forecast = 0.0              # mm (24h)

# Optimal thresholds (You can adjust for your greenhouse)
OPTIMAL_MIN_MOISTURE = 40
OPTIMAL_MAX_MOISTURE = 60
RAINFALL_THRESHOLD = 5               # If forecasted rain < 5 mm → irrigation needed

# Helper functions
def evaporation(temp, humidity):
    """Simple evaporation model: hotter + drier = more evaporation."""
    return max(0, 0.15 + 0.02 * (temp - 20) - 0.01 * (humidity - 50))

def rainfall_effect(rain):
    """Rain increases soil moisture slightly."""
    return 0.5 * rain

# Generating synthetic dataset: Data generation loop

In [3]:
timestamps = []
soil_list = []
temp_list = []
hum_list = []
rain_list = []
irrigation_list = []

current_time = start_time

for i in range(N):

    # 1) GENERATE TIMESTAMP
    timestamps.append(current_time)
    current_time += timedelta(minutes=interval_minutes)

    # 2) UPDATE ENV VARs
    temperature += np.random.uniform(-0.2, 0.2)
    humidity += np.random.uniform(-0.3, 0.3)

    # Daily rainfall forecast randomness
    if i % int(24*60/interval_minutes) == 0:
        rainfall_forecast = max(0, np.random.normal(3, 2))
    
    # Soil moisture dynamic 
    soil_moisture = soil_moisture \
                     - evaporation(temperature, humidity) \
                     + rainfall_effect(rainfall_forecast) \
                     + np.random.uniform(-0.1, 0.1)  # tiny noise

    soil_moisture = np.clip(soil_moisture, 10, 80)

    # 3) CONTROL LOGIC
    if soil_moisture < OPTIMAL_MIN_MOISTURE and rainfall_forecast < RAINFALL_THRESHOLD:
        irrigation = "HIGH"
    elif OPTIMAL_MIN_MOISTURE <= soil_moisture <= OPTIMAL_MAX_MOISTURE:
        irrigation = "MEDIUM"
    else:
        irrigation = "LOW"

    # Append data
    soil_list.append(soil_moisture)
    temp_list.append(temperature)
    hum_list.append(humidity)
    rain_list.append(rainfall_forecast)
    irrigation_list.append(irrigation)
       
# print(soil_list)
# print(temp_list)
# print(hum_list)
# print(rain_list)
# print(irrigation_list)

# Create dataframe and save it to csv

In [4]:
df = pd.DataFrame({
    "timestamp": timestamps,
    "soil_moisture": soil_list,
    "temperature": temp_list,
    "humidity": hum_list,
    "rainfall_forecast": rain_list,
    "irrigation_action": irrigation_list
})

# Save to CSV
df.to_csv("synthetic_greenhouse_dataset.csv", index=False)

df.head()

,timestamp,soil_moisture,temperature,humidity,rainfall_forecast,irrigation_action
0,2024-01-01 00:00:00,44.758475,24.126130,55.241811,0.0,MEDIUM
1,2024-01-01 00:10:00,44.640517,24.126732,55.015458,0.0,MEDIUM
2,2024-01-01 00:20:00,44.386942,24.008406,55.249518,0.0,MEDIUM
3,2024-01-01 00:30:00,44.229748,24.108533,55.326161,0.0,MEDIUM
4,2024-01-01 00:40:00,44.093483,23.959421,55.531438,0.0,MEDIUM


### Next up, we have fuzzification layer, which consisst of building membership functions and applying them to our dataframe.

* It is basically converting crisp numeric inputs into fuzzy linguistic values. For example, "soil moisture is $low$ with degree 0.82"    

# Fuzzification layer
## Membership functions

In [5]:
def trimf(x, a, b, c):
    
    if x<=a and x>=c:
        return 0.0

    elif a < x < b:
        return (x-a)/(b-a)
    
    elif b <= x < c:
        return (c-x)/(c-b)
    
    else:
        return 1.0

In [6]:
def trapmf(x, a, b, c, d):

    if x<=a or x>=d:
        return 0.0
    elif a < x < b:
        return (x-a)/(b-a)
    elif b <= x <= c:
        return 1.0
    elif c < x < d:
        return (d-x)/(d-c)

In [7]:
MF = {
    "soil_moisture": {
        "low":    lambda x: trapmf(x, 0, 0, 15, 30),
        "medium": lambda x: trimf(x, 15, 30, 45),
        "high":   lambda x: trapmf(x, 30, 45, 60, 60)
    },

    "temperature": {
        "cold":   lambda x: trimf(x, -5, 5, 15),
        "normal": lambda x: trimf(x, 10, 20, 30),
        "hot":    lambda x: trimf(x, 25, 35, 45),
    },

    "humidity": {
        "dry":    lambda x: trapmf(x, 0, 0, 20, 40),
        "normal": lambda x: trimf(x, 30, 50, 70),
        "humid":  lambda x: trapmf(x, 60, 80, 100, 100),
    },

    "rainfall_forecast": {
        "low":    lambda x: trapmf(x, 0, 0, 5, 15),
        "medium": lambda x: trimf(x, 10, 20, 30),
        "high":   lambda x: trapmf(x, 20, 40, 60, 60)
    }
}

In [8]:
df.head()

,timestamp,soil_moisture,temperature,humidity,rainfall_forecast,irrigation_action
0,2024-01-01 00:00:00,44.758475,24.126130,55.241811,0.0,MEDIUM
1,2024-01-01 00:10:00,44.640517,24.126732,55.015458,0.0,MEDIUM
2,2024-01-01 00:20:00,44.386942,24.008406,55.249518,0.0,MEDIUM
3,2024-01-01 00:30:00,44.229748,24.108533,55.326161,0.0,MEDIUM
4,2024-01-01 00:40:00,44.093483,23.959421,55.531438,0.0,MEDIUM


## Applying MF to form the fuzzified dataset

In [9]:
fuzzy_df = pd.DataFrame()

for feature, mfs in MF.items():
    for mf_name, mf_func in mfs.items():
        col_name = f"{feature}_{mf_name}"
        fuzzy_df[col_name] = df[feature].apply(mf_func)

In [10]:
fuzzy_df.shape

(2000, 12)

In [20]:
fuzzy_df.head()

,soil_moisture_low,soil_moisture_medium,soil_moisture_high,temperature_cold,temperature_normal,temperature_hot,humidity_dry,humidity_normal,humidity_humid,rainfall_forecast_low,rainfall_forecast_medium,rainfall_forecast_high
0,0.0,0.016102,0.983898,1.0,0.587387,1.0,0.0,0.737909,0.0,0.0,1.0,0.0
1,0.0,0.023966,0.976034,1.0,0.587327,1.0,0.0,0.749227,0.0,0.0,1.0,0.0
2,0.0,0.040871,0.959129,1.0,0.599159,1.0,0.0,0.737524,0.0,0.0,1.0,0.0
3,0.0,0.051350,0.948650,1.0,0.589147,1.0,0.0,0.733692,0.0,0.0,1.0,0.0
4,0.0,0.060434,0.939566,1.0,0.604058,1.0,0.0,0.723428,0.0,0.0,1.0,0.0


# The Rule Layer


As we finished with fuzzification layer, we start the Rule layer. The rule layer computes the strenghts of fuzzy rules. ANFIS rules come from fuzzy logic. A typical fuzzy rule looks like following, "IF temperature is HOT $\textbf{AND}$  Humidity is HIGH $\textbf{AND}$ RAINFALL is HEAVY $\textbf{THEN}$ output is $f(x)$, where f(x) is some linear function"

Each MF gives you a membership degree: 
* Temp_Hot(x) -> 0.82
* Humidity_High(x) -> 0.40
* Rainfall_Heavy(x) -> 0.10

A fuzzy $\textbf{AND}$ is implemented as a product in ANFIS, like probabilities od independent events:

$$w_i = \mu_{Temp, Hot}(x) * \mu_{Humidity, High}(x) * \mu_{Rainfall, Heavy}(x)$$

## MF 

In [12]:
soil_mfs = ["soil_moisture_low", "soil_moisture_medium", "soil_moisture_high"]
temp_mfs = ["temperature_cold", "temperature_normal", "temperature_hot"]
humidity_mfs = ["humidity_dry", "humidity_normal", "humidity_humid"]
rainfall_mfs = ["rainfall_forecast_low", "rainfall_forecast_medium", "rainfall_forecast_high"]

## Rule combinations

In [13]:
import itertools

rules = list(itertools.product(soil_mfs, temp_mfs, humidity_mfs, rainfall_mfs))
len(rules)

81

## Rule firing strenghts

In [14]:
rule_outputs = pd.DataFrame()

for i, (s, t, h, r) in enumerate(rules):
    rule_name = f"R{i+1}_{s}_{t}_{h}_{r}"
    rule_outputs[rule_name] = fuzzy_df[s] * fuzzy_df[t] * fuzzy_df[h] * fuzzy_df[r]

In [15]:
rule_outputs.shape

(2000, 81)

# Normalization Layer

$\overline{w_i} = w_i \Big/ (\sum_{j=1}^{\infty} w_j)$

### Where:
* $w_1, w_2, w_3, ..., w_{81}$ are the firing strenghts from Rule layer
* Because the next layer (consequent layer) needs weights that sum to 1 so it can compute a weighted average of the outputs.

In [16]:
# Sum of rule strenghts across all 81 rules (per row)
rule_sum = rule_outputs.sum(axis=1)

# Prevents division-by-zero in case all firing strengths are zero
rule_sum = rule_sum.replace(0, 1e-6)

# Normalized firing strengths
normalized_rules = rule_outputs.div(rule_sum, axis=0)

normalized_rules.head()

,R1_soil_moisture_low_temperature_cold_humidity_dry_rainfall_forecast_low,R2_soil_moisture_low_temperature_cold_humidity_dry_rainfall_forecast_medium,R3_soil_moisture_low_temperature_cold_humidity_dry_rainfall_forecast_high,R4_soil_moisture_low_temperature_cold_humidity_normal_rainfall_forecast_low,R5_soil_moisture_low_temperature_cold_humidity_normal_rainfall_forecast_medium,R6_soil_moisture_low_temperature_cold_humidity_normal_rainfall_forecast_high,R7_soil_moisture_low_temperature_cold_humidity_humid_rainfall_forecast_low,R8_soil_moisture_low_temperature_cold_humidity_humid_rainfall_forecast_medium,R9_soil_moisture_low_temperature_cold_humidity_humid_rainfall_forecast_high,R10_soil_moisture_low_temperature_normal_humidity_dry_rainfall_forecast_low,...,R72_soil_moisture_high_temperature_normal_humidity_humid_rainfall_forecast_high,R73_soil_moisture_high_temperature_hot_humidity_dry_rainfall_forecast_low,R74_soil_moisture_high_temperature_hot_humidity_dry_rainfall_forecast_medium,R75_soil_moisture_high_temperature_hot_humidity_dry_rainfall_forecast_high,R76_soil_moisture_high_temperature_hot_humidity_normal_rainfall_forecast_low,R77_soil_moisture_high_temperature_hot_humidity_normal_rainfall_forecast_medium,R78_soil_moisture_high_temperature_hot_humidity_normal_rainfall_forecast_high,R79_soil_moisture_high_temperature_hot_humidity_humid_rainfall_forecast_low,R80_soil_moisture_high_temperature_hot_humidity_humid_rainfall_forecast_medium,R81_soil_moisture_high_temperature_hot_humidity_humid_rainfall_forecast_high
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.380267,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.377237,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.369015,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.366395,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.360808,0.0,0.0,0.0,0.0


In [17]:
print("Any NaNs in rule_outputs?", rule_outputs.isna().any().any())
print("How many rows have sum=0?", (rule_outputs.sum(axis=1)==0).sum())

Any NaNs in rule_outputs? False
How many rows have sum=0? 0


In [18]:
normalized_rules.head()

,R1_soil_moisture_low_temperature_cold_humidity_dry_rainfall_forecast_low,R2_soil_moisture_low_temperature_cold_humidity_dry_rainfall_forecast_medium,R3_soil_moisture_low_temperature_cold_humidity_dry_rainfall_forecast_high,R4_soil_moisture_low_temperature_cold_humidity_normal_rainfall_forecast_low,R5_soil_moisture_low_temperature_cold_humidity_normal_rainfall_forecast_medium,R6_soil_moisture_low_temperature_cold_humidity_normal_rainfall_forecast_high,R7_soil_moisture_low_temperature_cold_humidity_humid_rainfall_forecast_low,R8_soil_moisture_low_temperature_cold_humidity_humid_rainfall_forecast_medium,R9_soil_moisture_low_temperature_cold_humidity_humid_rainfall_forecast_high,R10_soil_moisture_low_temperature_normal_humidity_dry_rainfall_forecast_low,...,R72_soil_moisture_high_temperature_normal_humidity_humid_rainfall_forecast_high,R73_soil_moisture_high_temperature_hot_humidity_dry_rainfall_forecast_low,R74_soil_moisture_high_temperature_hot_humidity_dry_rainfall_forecast_medium,R75_soil_moisture_high_temperature_hot_humidity_dry_rainfall_forecast_high,R76_soil_moisture_high_temperature_hot_humidity_normal_rainfall_forecast_low,R77_soil_moisture_high_temperature_hot_humidity_normal_rainfall_forecast_medium,R78_soil_moisture_high_temperature_hot_humidity_normal_rainfall_forecast_high,R79_soil_moisture_high_temperature_hot_humidity_humid_rainfall_forecast_low,R80_soil_moisture_high_temperature_hot_humidity_humid_rainfall_forecast_medium,R81_soil_moisture_high_temperature_hot_humidity_humid_rainfall_forecast_high
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.380267,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.377237,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.369015,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.366395,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.360808,0.0,0.0,0.0,0.0


In ANFIS / fuzzy inference, each normalized rule firing strength multiplies a linear consequent function:

$$f_i = p_is + q_it + r_ih + s_ir + t_i$$

Where, 
* s = soil moisture
* t = temperature
* h = humidity
* r = rainfall
* t = constant, error term



# Consequent Layer

In [23]:
df.head()

,timestamp,soil_moisture,temperature,humidity,rainfall_forecast,irrigation_action
0,2024-01-01 00:00:00,44.758475,24.126130,55.241811,0.0,MEDIUM
1,2024-01-01 00:10:00,44.640517,24.126732,55.015458,0.0,MEDIUM
2,2024-01-01 00:20:00,44.386942,24.008406,55.249518,0.0,MEDIUM
3,2024-01-01 00:30:00,44.229748,24.108533,55.326161,0.0,MEDIUM
4,2024-01-01 00:40:00,44.093483,23.959421,55.531438,0.0,MEDIUM


In [ ]:
#number of rules
num_rules = normalized_rules.shape[1]
num_params = 5 # soil_moisture, temp, hum, rainf, bias

# 1. Initiallize consequent params randomly
np.random.seed(42)
P = np.random.uniform(-1, 1, (num_rules, num_params))

# 2. Convert input dataframe to numpy matrix 
X1 = df['soil_moisture'].values
X2 = df['temperature'].values
X3 = df['humidity'].values
X4 = df['rainfall_forecast'].values

Compute the consequent output

In [ ]:
consequent_values = pd.DataFrame(index=df.index)

Loop through rules

In [ ]:
for i in range(num_rules):
    p, q, r, s, t = P[i]

    rule_name = normalized_rules.columns[i]

    consequent_values[rule_name] = (
        p * X1 +
        q * X2 +
        r * X3 + 
        s * X4 +
        t
    )

In [33]:
consequent_values

,R1_soil_moisture_low_temperature_cold_humidity_dry_rainfall_forecast_low,R2_soil_moisture_low_temperature_cold_humidity_dry_rainfall_forecast_medium,R3_soil_moisture_low_temperature_cold_humidity_dry_rainfall_forecast_high,R4_soil_moisture_low_temperature_cold_humidity_normal_rainfall_forecast_low,R5_soil_moisture_low_temperature_cold_humidity_normal_rainfall_forecast_medium,R6_soil_moisture_low_temperature_cold_humidity_normal_rainfall_forecast_high,R7_soil_moisture_low_temperature_cold_humidity_humid_rainfall_forecast_low,R8_soil_moisture_low_temperature_cold_humidity_humid_rainfall_forecast_medium,R9_soil_moisture_low_temperature_cold_humidity_humid_rainfall_forecast_high,R10_soil_moisture_low_temperature_normal_humidity_dry_rainfall_forecast_low,...,R72_soil_moisture_high_temperature_normal_humidity_humid_rainfall_forecast_high,R73_soil_moisture_high_temperature_hot_humidity_dry_rainfall_forecast_low,R74_soil_moisture_high_temperature_hot_humidity_dry_rainfall_forecast_medium,R75_soil_moisture_high_temperature_hot_humidity_dry_rainfall_forecast_high,R76_soil_moisture_high_temperature_hot_humidity_normal_rainfall_forecast_low,R77_soil_moisture_high_temperature_hot_humidity_normal_rainfall_forecast_medium,R78_soil_moisture_high_temperature_hot_humidity_normal_rainfall_forecast_high,R79_soil_moisture_high_temperature_hot_humidity_humid_rainfall_forecast_low,R80_soil_moisture_high_temperature_hot_humidity_humid_rainfall_forecast_medium,R81_soil_moisture_high_temperature_hot_humidity_humid_rainfall_forecast_high
0,35.460766,-11.245174,15.851515,-35.468778,-30.434978,11.702237,-53.394280,-26.391374,-85.991592,7.050030,...,-0.470215,-6.826149,55.981072,-104.649055,-52.670670,-9.971492,-18.374615,25.233831,48.518396,-15.882653
1,35.385882,-11.330320,15.814684,-35.405532,-30.367703,11.628153,-53.223144,-26.282229,-85.691646,7.002376,...,-0.543215,-6.780695,55.824993,-104.371854,-52.411730,-9.939548,-18.300449,25.175946,48.449350,-15.790917
2,35.451447,-10.879863,16.102238,-35.187055,-30.436415,11.561263,-53.403322,-26.580732,-85.716783,6.973906,...,-0.463238,-6.777432,55.963367,-104.245224,-52.624705,-9.732098,-18.553774,24.887955,48.112833,-15.682447
3,35.616710,-10.804077,16.398021,-35.122927,-30.575635,11.413647,-53.569784,-26.778487,-85.670293,6.888182,...,-0.589683,-6.719979,56.153665,-104.278358,-52.571806,-9.552890,-18.742277,24.696629,48.038297,-15.476256
4,35.611733,-10.428200,16.525023,-34.968102,-30.583943,11.431337,-53.679406,-26.969443,-85.757009,6.908281,...,-0.452075,-6.749374,56.213586,-104.192384,-52.821045,-9.454563,-18.892411,24.536468,47.784864,-15.485990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,15.289734,-44.824676,-36.192327,-57.714684,-13.096375,34.343001,-23.581094,13.234449,-92.101976,19.596214,...,13.553214,-13.916026,26.934866,-112.558775,-47.960193,-40.212833,18.648387,67.414046,80.703166,-44.604945
1996,15.104519,-44.816126,-36.411832,-57.670829,-12.941730,34.417204,-23.368347,13.402704,-91.964989,19.639487,...,13.651739,-13.941239,26.696384,-112.314861,-47.911199,-40.301748,18.799280,67.474151,80.630357,-44.711499
1997,15.128546,-44.527159,-36.339101,-57.615185,-12.972686,34.490070,-23.525610,13.233427,-92.209063,19.691037,...,13.799960,-13.994407,26.809395,-112.423500,-48.237244,-40.276689,18.676911,67.425795,80.514960,-44.796357
1998,15.152649,-44.695426,-36.336038,-57.656235,-12.986532,34.432879,-23.478577,13.296498,-92.096484,19.652466,...,13.694363,-13.958603,26.793135,-112.410872,-48.059402,-40.272912,18.716509,67.441345,80.596374,-44.725627


# Defuzzification Layer

For each sample:
1. Each rule produces a consequent value: $f_i(x)$
2. Each rule has a normalized firing strength: $\bar{w_i}$
3. Combine them using weighted average:
$$y = \sum_{i=1}^{\infty} \bar{w_i} * f_i(x)$$

This gives one crips output(irrigation amount, fan speed, nutrient dose, etc.)

In [36]:
rule_names = normalized_rules.columns

# Multiply normalized weights by consequent outputs(rule-by-rule)
weighted_ouptuts = normalized_rules[rule_names] * consequent_values[rule_names]

In [38]:
# Defuzzified crisp output = sum over all rules 
crisp_output = weighted_ouptuts.sum(axis=1)

In [40]:
# Convert to DataFrame
output_df = pd.DataFrame({"control output": crisp_output})

In [43]:
output_df

,control output
0,-15.028480
1,-14.675780
2,-13.555546
3,-12.970404
4,-12.314592
...,...
1995,32.686024
1996,32.217873
1997,32.756084
1998,32.537866


### So, at last we recieved certain values. They are not just random values, they are what the model tells you to use as an irrigation amount in liters/m^2. However, you might notice that there are negative values, which makes no sense, isn't it? Because the model has only one forward pass and it is unstable. This is what I came up so far. 

# Thank you!